In [1]:
import xarray as xr
import os
import glob
import numpy as np
import pandas as pd
import dask.dataframe as dd
from distributed import Client

/scratch1/mor582/miniconda3/envs/pyAODN/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/scratch1/mor582/miniconda3/envs/pyAODN/lib/python3.6/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
files = glob.glob('/scratch1/mor582/argo/**/*_prof.nc',recursive=True)

In [3]:
len(pd.DataFrame(files))

442

In [11]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:32907 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 20 Cores: 20 Memory: 134.95 GB


In [13]:
client.close()

In [5]:
from functools import partial

def process_mf(dsinput,levels,template):
    ds = xr.Dataset()
    pad =np.ones((len(dsinput.N_PROF), levels-len( dsinput.N_LEVELS))) *np.nan
    pading =xr.DataArray(pad,dims={'N_PROF','N_LEVELS'})
    for varname, da in template.data_vars.items():
        if 'N_LEVELS' in da.dims:   
            if varname in dsinput.data_vars:
                da = xr.concat([dsinput[varname],pading],dim='N_LEVELS')
            else:
                print(varname)
                da = xr.DataArray(np.ones((len(dsinput.N_PROF), levels)) *np.nan, name=varname, dims=['N_PROF','N_LEVELS'])
        elif varname in dsinput.data_vars:
            da = dsinput[varname]
        else:
            da = xr.DataArray(np.ones(len(dsinput.N_PROF))*np.nan , name=varname, dims=['N_PROF'])
            print(varname)
        if not ('HISTORY' in varname) and ('N_CALIB' not in da.dims) and ('N_PARAM' not in da.dims):
                ds[varname]= da
    ds =ds.chunk({'N_PROF':1000,'N_LEVELS':1000})
    return ds

template =xr.open_dataset(files[10])    
preproc = partial(process_mf,levels=2000,template=template)


In [6]:
files =[fs.open(file) for file in afloat]

NameError: name 'afloat' is not defined

In [7]:
ds1 = xr.open_mfdataset(files,preprocess=preproc,concat_dim='N_PROF',chunks={'N_PROF':1000,'N_LEVELS':1000})

In [8]:
ds1

<xarray.Dataset>
Dimensions:                   (N_LEVELS: 2000, N_PROF: 96734)
Dimensions without coordinates: N_LEVELS, N_PROF
Data variables:
    DATA_TYPE                 (N_PROF) object b'Argo profile    ' ... b'Argo profile    '
    FORMAT_VERSION            (N_PROF) object b'3.1 ' b'3.1 ' ... b'3.1 '
    HANDBOOK_VERSION          (N_PROF) object b'1.2 ' b'1.2 ' ... b'1.2 '
    REFERENCE_DATE_TIME       (N_PROF) object b'19500101000000' ... b'19500101000000'
    DATE_CREATION             (N_PROF) object b'20100226042203' ... b'20080816044514'
    DATE_UPDATE               (N_PROF) object b'20191008065233' ... b'20191015120815'
    PLATFORM_NUMBER           (N_PROF) object dask.array<shape=(96734,), chunksize=(352,)>
    PROJECT_NAME              (N_PROF) object dask.array<shape=(96734,), chunksize=(352,)>
    PI_NAME                   (N_PROF) object dask.array<shape=(96734,), chunksize=(352,)>
    CYCLE_NUMBER              (N_PROF) float64 dask.array<shape=(96734,), chunksize=(35

In [9]:
ds1 =ds1.chunk({'N_PROF':1000,'N_LEVELS':1000})

In [12]:
import zarr
import os
zarr_path =  '/scratch1/mor582/argo/csiro.zip' # MYSCRATCH is a Pawsey HPC environment variable

compressor = zarr.Blosc(cname='zstd', clevel=3, shuffle=2)
encoding = {vname: {'compressor': compressor} for vname in ds1.variables}
ds1.to_zarr(zarr_path,mode='w',encoding=encoding) #

/scratch1/mor582/miniconda3/envs/pyAODN/lib/python3.6/site-packages/xarray/conventions.py:172: SerializationWarning: variable None has data in the form of a dask array with dtype=object, which means it is being loaded into memory to determine a data type that can be safely stored on disk. To avoid this, coerce this variable to a fixed-size dtype with astype() before saving it.
  SerializationWarning)
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:40436 remote=tcp://127.0.0.1:32907>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:40438 remote=tcp://127.0.0.1:32907>


KeyboardInterrupt: 

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exc

In [18]:
ds1 = xr.open_zarr(zarr_path)
ds1.info()

xarray.Dataset {
dimensions:
	N_LEVELS = 2000 ;
	N_PROF = 11988 ;

variables:
	float64 CONFIG_MISSION_NUMBER(N_PROF) ;
		CONFIG_MISSION_NUMBER:conventions = 1...N, 1 : first complete mission ;
		CONFIG_MISSION_NUMBER:long_name = Unique number denoting the missions performed by the float ;
	float64 CYCLE_NUMBER(N_PROF) ;
		CYCLE_NUMBER:conventions = 0...N, 0 : launch cycle (if exists), 1 : first complete cycle ;
		CYCLE_NUMBER:long_name = Float cycle number ;
	|S2 DATA_CENTRE(N_PROF) ;
		DATA_CENTRE:conventions = Argo reference table 4 ;
		DATA_CENTRE:long_name = Data centre in charge of float data processing ;
	|S1 DATA_MODE(N_PROF) ;
		DATA_MODE:conventions = R : real time; D : delayed mode; A : real time with adjustment ;
		DATA_MODE:long_name = Delayed mode or real time data ;
	|S4 DATA_STATE_INDICATOR(N_PROF) ;
		DATA_STATE_INDICATOR:conventions = Argo reference table 6 ;
		DATA_STATE_INDICATOR:long_name = Degree of processing the data have passed through ;
	|S16 DATA_TYPE(N_PROF) 